# Storing Azure SPN in Azure Key Vault with Secret Management


As well as the local key vault that comes by default with the SecretManagement module, you can also register other key vaults to use with the module.

In this notebook, we will show how to add an Azure Key Vault to be used with the module and how to add and retrieve secrets.

## Get the modules

Lets make sure we have the modules we require. This code block will trust the PowerShell Gallery and install the AZ modules 

In [ ]:
if((Get-PsRepository -Name PSGallery).InstallationPolicy -ne 'Trusted'){
    Write-Output "The PowerShell Gallery is not trusted so I will trust it so that we can install the modules without interaction"
    Set-PsRepository -Name PSGallery -InstallationPolicy Trusted
} else {
    Write-Output "The PowerShell Gallery is trusted I will continue"
}
If((Get-Module Az.* -ListAvailable)){
    Write-Output "We have some AZ modules"
} else {
    Write-Output "We need to install the AZ modules"
    Install-Module -Name Az -AllowClobber -Scope CurrentUser
}
If((Get-Module dbatools -ListAvailable)){
    Write-Output "We have the dbatools module"
} else {
    Write-Output "We need to install the dbatools module"
    Install-Module -Name dbatools -Scope CurrentUser
}

## Connect to Azure

Next we will run `Connect-AzAccount`, you will need to open this link https://microsoft.com/devicelogin sign in and provide the authentication code

In [2]:
# Connect to Azure with a browser sign in token - 
Connect-AzAccount



Account                 SubscriptionName            TenantId                             Environmen


                                                                                         t


-------                 ----------------            --------                             ----------


mrrobsewell@outlook.com Microsoft Azure Sponsorship add02cc8-7eaf-4746-902a-53d0ceeff326 AzureCloud


## Choose the subscription

If you have multiple subscriptions, choose the one that your key vault is in.

In [31]:
# list subscriptions
Get-AzSubscription


Name                               Id                                   TenantId


----                               --                                   --------                   


Microsoft Azure Sponsorship        6d8f994c-9051-4cef-ba61-528bab27d213 add02cc8-7eaf-4746-902a-53…


TP Visual Studio Premium with MSDN 18107f13-584f-4e56-8824-31dfbfffaf7f add02cc8-7eaf-4746-902a-53…


Visual Studio Professional         dd6620e8-486c-4bf7-8c62-115d3e04ea6c add02cc8-7eaf-4746-902a-53…


MVP - Visual Studio Enterprise     71df03d4-33d9-40ca-8924-a69d8ae8924a add02cc8-7eaf-4746-902a-53…


Free Trial                         e8a3b955-9a1e-46ac-a9eb-c0cd80b20980 add02cc8-7eaf-4746-902a-53…


In [33]:
# Set subscription by Name
$AzureSubscription = Set-AzContext -SubscriptionName "Microsoft Azure Sponsorship"

Name               : Microsoft Azure Sponsorship (6d8f994c-9051-4cef-ba61-528bab27d213) - 


                     mrrobsewell@outlook.com


Account            : mrrobsewell@outlook.com


Environment        : AzureCloud


Subscription       : 6d8f994c-9051-4cef-ba61-528bab27d213


Tenant             : add02cc8-7eaf-4746-902a-53d0ceeff326


TokenCache         : Microsoft.Azure.Commands.Common.Authentication.Core.ProtectedFileTokenCache


VersionProfile     : 


ExtendedProperties : {}


## Register your Azure Key Vault with Secret Management Module

For the Secret Management Module to manage the Azure Key Vault, you first need to register it. 

Ensure that you have permissions to connect by following the details in the network security documentation https://docs.microsoft.com/en-us/azure/key-vault/general/network-security and the secure access documentation https://docs.microsoft.com/en-us/azure/key-vault/general/secure-your-key-vault

Then you can run `Register-SecretVault` You need to provide the local name for the key vault, the module name `Az.KeyVault`, and a `VaultParameters` hashtable with the KeyVault name and the Azure Subscription ID. You can register other types of Key Vaults to the Secret Management module, they will require different values for the `VaultParameters` parameter.

In [ ]:
$KeyVaultName = 'beard-key-vault'
Register-SecretVault -Name BeardKeyVault -ModuleName Az.KeyVault -VaultParameters @{ AZKVaultName = $KeyVaultName; SubscriptionId = $AzureSubscription.Subscription }

## Create an SPN in Azure

With the Az modules you can create a new Service Principal Name and then store the appid and the secret in the key vault

In [4]:
$sp = New-AzADServicePrincipal -DisplayName BeardElasticPoolSPN
Set-Secret -Vault BeardKeyVault -Name BeardElasticPoolSPNApplicationID -Secret (ConvertTo-SecureString  $sp.ApplicationId.Guid -AsPlainText)
Set-Secret -Vault BeardKeyVault -Name BeardElasticPoolSPNSecret -Secret $sp.secret
Set-Secret -Vault BeardKeyVault -Name BeardElasticPoolSPNSecret -Secret bKjNF4-M7w004JrnTlZnvY.C-fAyg1-whd

In [43]:
Get-SecretVault -Name BeardKeyVault
Get-SecretInfo -Vault BeardKeyVault -Name BeardElasticPoolSPNApplicationID 
Get-SecretInfo -Vault BeardKeyVault -Name BeardElasticPoolSPNSecret

Name          ModuleName  ImplementingType


----          ----------  ----------------


BeardKeyVault Az.KeyVault 


Name      : BeardElasticPoolSPNApplicationID


Type      : SecureString


VaultName : BeardKeyVault


Name      : BeardElasticPoolSPNSecret


Type      : SecureString


VaultName : BeardKeyVault


In [69]:
Set-Secret -Vault BeardKeyVault -Name BeardElasticPoolSPNSecret -Secret (ConvertTo-SecureString  'bKjNF4-M7w004JrnTlZnvY.C-fAyg1-whd' -AsPlainText) 

In [78]:
$AzureSql = 'beard-elasticsql-dev.database.windows.net'
$appid = Get-Secret -Vault BeardKeyVault -Name BeardElasticPoolSPNApplicationID -AsPlainText
$Clientsecret = Get-Secret -Vault BeardKeyVault -Name BeardElasticPoolSPNSecret
$credential = New-Object System.Management.Automation.PSCredential ($appid,$Clientsecret)
$appid 
$Clientsecret
$credential.GetNetworkCredential().Password
Invoke-DbaQuery -SqlInstance $AzureSql -Database devbeardsqldma  -SqlCredential $credential -Query "Select 1 as one" -Verbose


8851588b-3041-4024-8fac-812b07199258


System.Security.SecureString


bKjNF4-M7w004JrnTlZnvY.C-fAyg1-whd


VERBOSE: [08:47:58][Connect-DbaInstance] Connecting to Data Source=TCP:beard-elasticsql-dev.database.windows.net,1433;Initial Catalog=devbeardsqldma;User ID=8851588b-3041-4024-8fac-812b07199258;Password=********;MultipleActiveResultSets=False;Connect Timeout=30;Encrypt=True;TrustServerCertificate=False;Application Name="dbatools PowerShell module - dbatools.io"


VERBOSE: [08:47:58][Connect-DbaInstance] Connecting to Azure: beard-elasticsql-dev.database.windows.net


In [75]:
Get-Error

Exception             : 


    Type           : System.Exception


    Message        : Login failed for user '8851588b-3041-4024-8fac-812b07199258'.


    InnerException : 


        Type               : System.Data.SqlClient.SqlException


...

C:\Users\mrrob\OneDrive\Documents\PowerShell\Modules\dbatools\1.0.113\allcommands.ps1: line 83443


                        at Invoke-DbaQuery<Process>, 


C:\Users\mrrob\OneDrive\Documents\PowerShell\Modules\dbatools\1.0.113\allcommands.ps1: line 47464


                        at <ScriptBlock>, <No file>: line 8


PipelineIterationInfo : 
